To-do:
- Run connectivity on 5 rest subjects (adjust parallelization as needed)
- Assess reliability (permutation tests if not obvious)
- Run averaging and SRM code
- Figure out task decoding and write task_decoding.py


To think about:
- Searchlights instead of parcels as connectivity targets, or as SRM boundaries
- Distance as SRM penalty instead of parcelwise
- DiFuMo atlas instead of Schaefer?
- Reliability.py: permutation tests?
- Write task_decoding.py

In [ ]:
import glob
import numpy as np
import nibabel as nib
from nilearn import datasets
from nilearn.image import resample_img, math_img
from nilearn.maskers import NiftiMasker, NiftiLabelsMasker
from joblib import Parallel, delayed
from math import tanh
from scipy.stats import pearsonr

In [2]:
from sherlock.sherlock_connectivity import load_data_one_session, get_parcellation, compute_fc_one_session

In [3]:
f = 'data/rest/sub-s03_ses-02_task-rest_run-1_space-MNI_desc-optcomDenoised_bold.nii.gz'
c = 'data/confounds/sub-s03_ses-02_task-rest_run-1_desc-confounds_timeseries.tsv'


parcel_labels, parcel_map, parcel_mask = get_parcellation(schaefer_n_rois = 400, resample_target = nib.load(f)) 
voxel_data, parcel_data = load_data_one_session(f,c, parcel_labels, parcel_map, parcel_mask)

c:\Users\ciyer\miniconda3\envs\lab\Lib\site-packages\joblib\memory.py:349: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


In [ ]:
# TO-DO!!!

# 1. make it faster
# 2. exclude ones already done
# 3. run it again

In [ ]:
vtest = voxel_data[:, 0:100]
ptest = parcel_data[:, 0:40]

In [ ]:
connectome = compute_fc_one_session(vtest, ptest, zscore=True)
print(connectome.shape)
connectome[0:5, 0:5]

(100, 40)


array([[0.75871411, 0.75510484, 0.73408039, 0.7340864 , 0.72387146],
       [0.75932942, 0.75591503, 0.73147384, 0.73164462, 0.72159261],
       [0.74215049, 0.73570163, 0.74995086, 0.74990275, 0.73393837],
       [0.73890848, 0.73206347, 0.75625313, 0.75603601, 0.73635248],
       [0.759307  , 0.75491649, 0.73327964, 0.73356438, 0.7231754 ]])

In [ ]:
def correlate_one_pair(i, j):
    return tanh(pearsonr(vtest[:,i], ptest[:,j])[0])
def correlate_one_voxel(i):
   return [correlate_one_pair(i,j) for j in range(ptest.shape[1])]

ctest = Parallel(n_jobs = 4)(
    delayed(correlate_one_voxel)(i) for i in range(vtest.shape[1])
)

ctest = np.vstack(ctest)
ctest[0:5,0:5]

array([[0.75871411, 0.75510484, 0.73408039, 0.7340864 , 0.72387146],
       [0.75932942, 0.75591503, 0.73147384, 0.73164462, 0.72159261],
       [0.74215049, 0.73570163, 0.74995086, 0.74990275, 0.73393837],
       [0.73890848, 0.73206347, 0.75625313, 0.75603601, 0.73635248],
       [0.759307  , 0.75491649, 0.73327964, 0.73356438, 0.7231754 ]])

In [ ]:
def correlate_one_voxel(i):
   corrs = np.corrcoef(vtest[:,i], ptest, rowvar=False)[0, 1:]
   return [tanh(c) for c in corrs]

ctest = Parallel(n_jobs = 4)(
    delayed(correlate_one_voxel)(i) for i in range(vtest.shape[1])
)

ctest = np.vstack(ctest)
ctest[0:5,0:5]

array([[0.75871411, 0.75510484, 0.73408039, 0.7340864 , 0.72387146],
       [0.75932942, 0.75591503, 0.73147384, 0.73164462, 0.72159261],
       [0.74215049, 0.73570163, 0.74995086, 0.74990275, 0.73393837],
       [0.73890848, 0.73206347, 0.75625313, 0.75603601, 0.73635248],
       [0.759307  , 0.75491649, 0.73327964, 0.73356438, 0.7231754 ]])